In [1]:
import pandas as pd
import pickle
from PopSynthesis.Methods.IPSF.const import data_dir, POOL_SIZE, processed_dir
from PopSynthesis.Methods.IPSF.CSP.operations.convert_seeds import convert_seeds_to_pairs, pair_states_dict
from PopSynthesis.Methods.IPSF.utils.pool_utils import create_pool

In [2]:
# get the data
hh_marg = pd.read_csv(data_dir / "hh_marginals_ipu.csv", header=[0, 1])
hh_seed = pd.read_csv(data_dir / "hh_sample_ipu.csv").drop(columns=["sample_geog"])
pp_seed = pd.read_csv(data_dir / "pp_sample_ipu.csv").drop(columns=["sample_geog"])
with open(processed_dir / "dict_hh_states.pickle", "rb") as handle:
    hh_att_state = pickle.load(handle)
with open(processed_dir / "dict_pp_states.pickle", "rb") as handle:
    pp_att_state = pickle.load(handle)

In [3]:
hh_marg = hh_marg.drop(columns=hh_marg.columns[hh_marg.columns.get_level_values(0)=="sample_geog"][0])
# vars
rela_col = "relationship"
id_col = "serialno"
main_rela = "Main"

In [4]:
# process seed
seed_pairs = convert_seeds_to_pairs(hh_seed, pp_seed, id_col, rela_col, main_rela)

In [5]:
# create pools
pools_ref = {}
for pair_name, pair_seed in seed_pairs.items():
    name1, name2 = pair_name.split("-")
    ori_states_1 = hh_att_state if name1 == "HH" else pp_att_state
    ori_states_2 = pp_att_state # because the second one always people
    processed_states_ref = pair_states_dict(ori_states_1, ori_states_2, name1, name2)
    # we only need matching columns that we wish to process for BN
    # this excludes relationship and ids
    assert set(processed_states_ref.keys()) <= set(pair_seed.columns)
    to_filter_col = list(processed_states_ref.keys())
    if name1 == "HH":
        # special case to add the rela cols
        to_filter_col += list(pp_seed[rela_col].unique())
    filtered_seed = pair_seed[to_filter_col]
    pools_ref[pair_name] = create_pool(filtered_seed, state_names=processed_states_ref, pool_sz=POOL_SIZE)

Learn BN


  0%|          | 0/1000000 [00:00<?, ?it/s]

Doing the sampling


  0%|          | 0/18 [00:00<?, ?it/s]

Learn BN


...

  0%|          | 0/10 [00:00<?, ?it/s]

Learn BN


  0%|          | 0/1000000 [00:00<?, ?it/s]

Doing the sampling


  0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
with open(processed_dir / "dict_pool_pairs.pickle", "wb") as handle:
    pickle.dump(pools_ref, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
pools_ref

{'HH-Main':                  dwelltype         owndwell      hhinc hhsize totalvehs  \
 0           Separate House      Fully Owned  1250-1499      4         2   
 1        Terrace/Townhouse      Fully Owned    400-499      2         1   
 2        Flat or Apartment     Being Rented  1750-1999      3         1   
 3           Separate House      Fully Owned    300-399      2         2   
 4                    Other      Fully Owned    800-999      1         1   
 ...                    ...              ...        ...    ...       ...   
 4999995     Separate House      Fully Owned    800-999      1         1   
 4999996     Separate House      Fully Owned  1250-1499      2         2   
 4999997     Separate House      Fully Owned    150-299      2         2   
 4999998     Separate House      Fully Owned  2000-2499      2         2   
 4999999     Separate House  Being Purchased    500-649      2         1   
 
                  persinc    age sex     nolicence anywork  Main  Spouse  \